## Parker Dunn
# Exploring Ruby player tracking data from Catapult

### Started on 7 March 2022
### Updates
* March 7 -> getting things setup; mostly working in `datapreparation.py`
*

In [9]:
# IMPORTS
import pandas as pd
import os
from datapreparation import *

# OTHER SETUP

dir_list = os.listdir("./raw_data")

# Used for debugging ... this is not the problem
# print(dir_list, type(dir_list))
# for file in dir_list:
#     print(file)



# Loading the Data

full_game_player_dds = []
for file in dir_list:
    if file[-4:]==".csv":
        full_game_player_dds.append(load_to_dask_df(file))

# debugging above
print(len(full_game_player_dds)) # <- returned 15 ... also not the issue

for player in full_game_player_dds:
    print(player.athlete_id.unique().compute())

15
0    214f9c05-cae3-4291-9a84-4d144755de90
Name: athlete_id, dtype: object
0    99f9cd94-e362-4d48-813c-7ea752d69fd1
Name: athlete_id, dtype: object
0    64c57ea9-ce79-421c-b38b-2a4bd5613053
Name: athlete_id, dtype: object
0    21a59223-121b-4418-87ce-f4985a3466fa
Name: athlete_id, dtype: object
0    20414b56-6832-4e4c-8750-1e28a5a15e55
Name: athlete_id, dtype: object
0    ca83ecda-125b-4d01-984a-bd0c96ec672a
Name: athlete_id, dtype: object
0    97977419-7aca-41d0-97f8-8a825600d1b4
Name: athlete_id, dtype: object
0    9d5dc314-5778-4510-9f60-a2e44c4acbaf
Name: athlete_id, dtype: object
0    eafb3b02-8f41-4723-a383-6d88116deaac
Name: athlete_id, dtype: object
0    dc9789ed-25d5-4803-b053-bbf97f6af37e
Name: athlete_id, dtype: object
0    2788f453-34e8-4450-ac8d-bbd467c3abc0
Name: athlete_id, dtype: object
0    b05122c9-25dc-4924-a3b5-5467f3d2d737
Name: athlete_id, dtype: object
0    40402c38-109c-4a96-93a6-f8760e0de30b
Name: athlete_id, dtype: object
0    9c44cfc9-f322-47b1-863c-af29ea